<a href="https://colab.research.google.com/github/aashish2000/Emotion-Detection/blob/master/Simple%20Neural%20Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [0]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, GlobalAveragePooling2D, BatchNormalization, Conv2D
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import MaxPooling2D, Activation, Dropout
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)
earlystop = EarlyStopping(patience=2)
callbacks = [earlystop, learning_rate_reduction]

batch_size=32
momentum=0.9
epochs=10
epoch_size=400
lr=0.005
wd=0.0005

train_datagen = ImageDataGenerator(rescale=1/255, rotation_range=90)
validation_datagen = ImageDataGenerator(rescale=1/255, rotation_range=90)
test_datagen = ImageDataGenerator(rescale=1./255,rotation_range=90)
nb_train_samples = 28709
nb_validation_samples = 3589

In [29]:
train_generator = train_datagen.flow_from_directory(
        "/content/drive/My Drive/fer_images/Training",  # This is the source directory for training images
        target_size=(48, 48),  # All images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True)

validation_generator = validation_datagen.flow_from_directory(
        "/content/drive/My Drive/fer_images/PrivateTest",  # This is the source directory for training images
        target_size=(48, 48),  # All images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True)
test_generator = train_datagen.flow_from_directory(
        "/content/drive/My Drive/fer_images/PublicTest",  # This is the source directory for training images
        target_size=(48, 48),  # All images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True)
filenames = test_generator.filenames
nb_test_samples = len(filenames)

Found 28709 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.


In [30]:
model= Sequential()

model.add(Conv2D(16, (3,3), input_shape=(48, 48, 3),use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.2))

model.add(Conv2D(32, (3,3),use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3,3),use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.2))

model.add(Conv2D(256, (3,3), use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(512,activation='relu'))
model.add(Dense(7,activation='softmax'))

model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 46, 46, 16)        432       
_________________________________________________________________
batch_normalization_12 (Batc (None, 46, 46, 16)        64        
_________________________________________________________________
activation_11 (Activation)   (None, 46, 46, 16)        0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 23, 23, 16)        0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 23, 23, 16)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 21, 21, 32)        4608      
_________________________________________________________________
batch_normalization_13 (Batc (None, 21, 21, 32)       

In [31]:
model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit_generator(
      train_generator,
      steps_per_epoch=nb_train_samples // batch_size,
      validation_data=validation_generator, 
      validation_steps=nb_validation_samples // batch_size,
      epochs=epochs,
      callbacks=callbacks)

score = model.evaluate_generator(validation_generator,nb_validation_samples//batch_size)

print(" Total Validation Accuracy: ", len(validation_generator.filenames))
print("Loss: ", score[0], "Accuracy: ", score[1])

model.save("/content/drive/My Drive/emotion-models/4layer_model.h5")

Epoch 1/10
 18/897 [..............................] - ETA: 3:17:25 - loss: 2.0727 - acc: 0.1944

KeyboardInterrupt: ignored